In [1]:
import bz2
import functools
import pickle

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import wikipedia2vec

In [2]:
with bz2.open("../enwiki_20180420_100d.pkl.bz2", "rb") as f:
    wiki2vec = wikipedia2vec.Wikipedia2Vec.load(f)

/home/ocifka/miniconda3/envs/wiki2vec/lib/python3.8/contextlib.py:113: UserWarning: "<bz2.BZ2File object at 0x7f444c61eee0>" is not a raw file, mmap_mode "c" flag will be ignored.
  return next(self.gen)


In [3]:
individuals = pd.read_parquet("individuals_simon.parquet")

In [4]:
# Manually defined cases that are not resolved automatically
TAXON_MAP = {
    "Mountain caribou": "Rangifer tarandus"
}

In [5]:
taxon2emb = {}

for taxon in individuals["species"].unique():
    print(taxon, end=" ")
    entity = None
    if taxon in TAXON_MAP:
        entity = wiki2vec.get_entity(TAXON_MAP[taxon])
        print("->", TAXON_MAP[taxon], end=" ")
    else:
        entity = wiki2vec.get_entity(taxon)
    if entity is None:
        print("-> FAIL")
    else:
        print("->", entity)
        taxon2emb[taxon] = wiki2vec.get_vector(entity)

Wild boar -> <Entity Wild boar>
Blue wildebeest -> <Entity Blue wildebeest>
Plains zebra -> <Entity Plains zebra>
African elephant -> <Entity African elephant>


In [6]:
nan_emb = next(iter(taxon2emb.values())) * np.nan

In [7]:
individual2emb = {
    ind: taxon2emb[taxon] if taxon in taxon2emb else nan_emb
    for ind, taxon in individuals[["id", "species"]].drop_duplicates().itertuples(index=False)
}

with open("individual2taxon_emb_simon.pickle", "wb") as f:
    pickle.dump(individual2emb, f)